# import and download

In [1]:
import gc
from pathlib import Path
import cudf
import pandas as pd
import numpy as np

from tqdm import tqdm 
# from sklearn.preprocessing import  MinMaxScaler
import pickle

In [3]:

def f():
    for i in range(10):
        print(i)

In [4]:
%%debug
f()

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()



ipdb>  s


--Call--
> /tmp/ipykernel_17/3846476012.py(1)f()
----> 1 def f():
      2     for i in range(10):
      3         print(i)



ipdb>  n


> /tmp/ipykernel_17/3846476012.py(2)f()
      1 def f():
----> 2     for i in range(10):
      3         print(i)



...

ipdb>  n


1
> /tmp/ipykernel_17/3846476012.py(2)f()
      1 def f():
----> 2     for i in range(10):
      3         print(i)



ipdb>  p i


1


ipdb>  q


In [ ]:
continue to clean notebook , all remarks and the recall generatoe loops
Clean rules
train on valid week +1 like the last cadidtes is 0 and no -1 ts2 = TopSales2(trans, n_weeks=10, n_articles=10, valid_week=valid_week+1)
check age binning running well (its category) maybe can run in cudf if change to int
candidates['customer_id'] = candidates['customer_id'].astype('int32') -added afetr cusid mapping ---added cusid customer_id still 64

remove columns with low feature importance
down sampling negative sample (i.e neg = 30*pos, or winner sulotion post 1st place)
replace description with word2vec embadding
    
# avoid chnging to pandas as much as possible, check if .values can solve like in this case:
preds = preds.to_pandas() #check if cuold avoid ad test in CV probably can avoid 
vc = trans.article_id.value_counts()
for i in vc.index.values[1000:1010]: #with no .values we get error to change to pandas
    print(i)
 
Furthur study:
validation test model analysis (trian vs ble overfit bla)  - output the score interations - shuold have it in code somewhere hidden

plot the desicon tree

#-------trrials to save the model:
# import lightgbm
# ranker.booster_.save_model('mode.txt')
or
# ranker.save_model('mode.txt')
# bst = lightgbm.Booster(model_file='./mode.txt')
or 
from sklearn.externals import joblib
# save model
joblib.dump(lgbmodel, 'lgb.pkl')
# load model
gbm_pickle = joblib.load('lgb.pkl')
#-------trrials to save the model:


In [ ]:
Future todo to improve score:
    1 optimize model with parameter tuning
    2 make addtional model and shuffle
    3 add features to customer, articles:
        Add feature mean channel for customer and for article
        start to work on #department_name, #product_type_name features or more recalls - top features
    unsolved- trend history is better but score split betwwen name and score and total performace is lower
    try differnt parametr for final submission, like more candidates for top sales 
    #checking after label count, to see if can improve the duplications in diffrent weeks (dont think the valid week is strong enough and might confuse the model, maybe cancel duplication leav only highest week?)

    
    Continuu to experimanet is strtgy ration:
    try addtional simple one instesd of pairs (same name? last week purchase? based on high features tree score)
    add addtional stragy that generate like history top n bought by user? like last x purchesed?
    item_cf
    customer_cf
    article mean price and recommnd each customer top sales in customer price category
# -H&M RAPIDS Article2Vec - short code finds similiar items by vector the description https://www.kaggle.com/code/aerdem4/h-m-rapids-article2vec/notebook
# - SIMILIAR PICTURE:
#  FROM EMBEDDING AND knn https://www.kaggle.com/code/joelqv/h-m-product-similarity-1-embeddings-knn/notebook
# - embedding from torch NN? 
#check also this guy: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/324185
# Faster and shorter basket anylyssis: https://www.kaggle.com/code/titericz/article-id-pairs-in-3s-using-cudf

future to improve code:
    Build rule collector function
    maybe add thershhold for positive rate for each rule (run smaller articles number if not meet)

# Reducing memory

In [ ]:
#old reduction transactions
id_to_index_dict = dict(zip(customers1["customer_id"].to_pandas(), customers1.index))
index_to_id_dict = dict(zip(customers1.index, customers1["customer_id"].to_pandas()))
train["customer_id"] = train["customer_id"].map(id_to_index_dict).astype('int32')
train['article_id'] = train['article_id'].astype('int32')
train['price'] = train['price'].astype('float32')
train['sales_channel_id'] = train['sales_channel_id'].astype('int8')


#customer reducing memory old version 
def dictmap(df, col, typ='int32'):
    dictval = df[col].unique()
    cols_to_idx_dict = dict(zip(dictval.to_pandas(), dictval.index))
    idx_to_cols_dict = dict(zip(dictval.index, dictval.to_pandas()))
    df[col] = df[col].map(cols_to_idx_dict).astype(typ)
    return cols_to_idx_dict, idx_to_cols_dict

postal_to_idx_dict, idx_to_postal_dict = dictmap(customers, 'postal_code')
# custID_to_idx_dict, idx_to_custID = dictmap(customers, 'customer_id')
customers['customer_id'] = customers.customer_id.map(id_to_index_dict).astype('int32') 
fnf_to_idx_dict, idx_to_fnf = dictmap(customers, 'fashion_news_frequency', typ='int8') #reduce more than category 
cms_to_idx_dict, idx_to_cms = dictmap(customers, 'club_member_status', typ='int8')
customers['FN'] = customers.FN.astype('int8')
customers['Active'] = customers.Active.astype('int8')
customers['age'] = customers.age.astype('int8')

In [ ]:
#new final version  - currenlty loading it to save time
#initial inspiration from Radek H&M startes packs (he use with top samples)
#https://github.com/radekosmulski/personalized_fashion_recs/blob/main/helper_functions.ipynb
#old way here under for age, active, FN using map and int8 can reduce more memory (maybe add condition if ints not obj transfer to 'int')
#Check null cases
#Maybe can reduce in same cases by half if mapping using negative numbers or u_int(also the mappping has no warning for uniques) - not sure about it
# Standart memeory reduction by type, min/max vlaues cuold be found in this link: #https://www.kaggle.com/code/sergiomananas/memory-reduction-script
# whole class need to be refactor (patch with the filter function to avoid building un-necerary dicts, also amany exception in trans, also probably articles can just drop all descriprion that have code)

from sklearn.base import BaseEstimator, TransformerMixin
class DataHelper(BaseEstimator, TransformerMixin):
    """
    missing docstring
    """
    def __init__(self, min_uniques=10, unchanged_cols=[]):
        self.min_uniques = min_uniques
        self.mapdict = {}
        self.unchanged_cols = unchanged_cols

        
    def fit(self, df):
        for col in (x for x in df.columns if self._cols_filter(x, df)):
            col_unique = df[col].unique()
            self.mapdict[f'{col}_to_idx_dict'] = dict(zip(col_unique.to_pandas(), col_unique.index))
            self.mapdict[f'idx_to_{col}_dict'] = dict(zip(col_unique.index,col_unique.to_pandas()))
        return self

    def _cols_filter(self, x, df):
        col_condition = ((df[x].nunique()>self.min_uniques)
                       and (x not in self.unchanged_cols)
                       )
        return col_condition
    
    def _min_int_for_dict(self, num):
        if num <= (2 ** 8)/2:
            typ = 'int8'
        elif num <= (2 ** 16)/2:
            typ = 'int16'
        elif num <= (2 ** 32)/2:
            typ = 'int32'
        else:
            typ = 'int64'   
        return typ
    
    def transform(self, df):
        start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
        for col in df.columns:
            if col in self.unchanged_cols:
                continue 
            if df[col].nunique() > self.min_uniques:
                typ = self._min_int_for_dict(df[col].nunique())
                df[col] = df[col].map(self.mapdict[f'{col}_to_idx_dict']).astype(typ)
            else:
#                 df[col] = df[col].astype('category')  shuold be faster need to update
                df[col] = pd.Categorical(df[col].to_pandas())
        end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
        percent = 100 * (start_mem - end_mem) / start_mem
        print(f'Mem. usage decreased from {start_mem:5.2f} Mb to {end_mem:5.2f} Mb ({percent:.1f}% reduction)')
        return df

#loading original dataset
path = Path('../input/h-and-m-personalized-fashion-recommendations')
customers = cudf.read_csv(path/'customers.csv', low_memory=False)
articles = cudf.read_csv(path/'articles.csv', low_memory=False)
trans = cudf.read_csv(path/'transactions_train.csv', low_memory=False)#, nrows=1_200_00)

#reducing memory and adding week no.
DHC = DataHelper(100)
customers = DHC.fit_transform(customers)

def is_reduntant(x):
    return ('id' in x)|('no' in x)|('code' in x)

redundant_cols = list(filter(is_reduntant, articles.columns))[1:]
articles.drop(columns=redundant_cols, inplace=True)

DHA = DataHelper(300)
articles = DHA.fit_transform(articles)

# Write the whole thing in modern python
start_mem = trans.memory_usage(deep=True).sum() / 1024 ** 2
DHT = DataHelper(300 ,['t_dat', 'customer_id', 'article_id', 'price'])
trans = DHT.fit_transform(trans)

trans['price'] = trans.price.astype('float32')
trans['t_dat'] = pd.to_datetime(trans.t_dat.to_pandas())

# typ = DHT._min_int_for_dict(trans['article_id'].nunique()) wont work after trans sample reduce
trans['customer_id'] = trans['customer_id'].map(DHC.mapdict['customer_id_to_idx_dict']).astype('int32')

# typ = DHT._min_int_for_dict(trans['article_id'].nunique())
trans['article_id'] = trans['article_id'].map(DHA.mapdict['article_id_to_idx_dict']).astype('int32')

end_mem = trans.memory_usage(deep=True).sum() / 1024 ** 2
percent = 100 * (start_mem - end_mem) / start_mem
print(f'Mem. usage Total decreased from {start_mem:5.2f} Mb to {end_mem:5.2f} Mb ({percent:.1f}% reduction)')

#later addtion probably shuold do it after the data load and let datahelper handle it
trans['week'] = (trans.t_dat.max() - trans.t_dat).dt.days // 7
trans['week'] = trans['week'].astype('int8')

#saving all files and dicts

# customers.to_csv('customers_r.csv')
# articles.to_csv('articles_r.csv')
# trans.to_csv('trans_r.csv', chunksize=1_000_000) 

# import pickle

# customer_id_to_idx_dict = DHC.mapdict['customer_id_to_idx_dict']
# idx_to_customer_id_dict = DHC.mapdict['idx_to_customer_id_dict']

# with open('customer_id_to_idx_dict.pkl', 'wb') as handle:
#     pickle.dump(customer_id_to_idx_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('idx_to_customer_id_dict.pkl', 'wb') as handle:
#     pickle.dump(idx_to_customer_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# article_id_to_idx_dict = DHA.mapdict['article_id_to_idx_dict']
# idx_to_article_id_dict = DHA.mapdict['idx_to_article_id_dict']

# with open('article_id_to_idx_dict.pkl', 'wb') as handle:
#     pickle.dump(article_id_to_idx_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('idx_to_article_id_dict.pkl', 'wb') as handle:
#     pickle.dump(idx_to_article_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
%%time
path = Path('../input/h-and-m-personalized-fashion-recommendations') #for test
path_reduce = Path('/kaggle/input/hmdata')
customers = cudf.read_csv(path_reduce/'customers_r.csv', index_col=0, low_memory=False)
articles = cudf.read_csv(path_reduce/'articles_r.csv', index_col=0, low_memory=False)
trans = cudf.read_csv(path_reduce/'trans_r.csv', index_col=0, low_memory=False)

# customers memory reduction
cus_cols = ['FN', 'Active', 'club_member_status','fashion_news_frequency', 'age']
for col in cus_cols:
    customers[col] = customers[col].astype('category')
customers['customer_id'] = customers['customer_id'].astype('int32') 
customers['postal_code'] = customers['postal_code'].astype('int32') 
bins = list(range(16,81,4))+ [100]
names = list(range(len(bins)-1))
customers = customers.to_pandas()
customers['age_group'] = pd.cut(customers['age'], bins, labels=names, right=False)
customers  = cudf.from_pandas(customers)
#articles memory reduction
art_to_cat =  ['product_type_name', 'product_group_name','graphical_appearance_name', 'colour_group_name', 
              'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name', 'index_name', 
              'index_group_name',  'section_name', 'garment_group_name']
art_to_int = ['article_id', 'prod_name', 'detail_desc']
for col in art_to_cat:
    articles[col] = articles[col].astype('category')
for col in art_to_int:
    articles[col] = articles[col].astype('int32') 
#trans memort reduction
trans['price'] = trans.price.astype('float32')
trans['t_dat'] = cudf.to_datetime(trans['t_dat'])
trans['customer_id'] = trans['customer_id'].astype('int32')
trans['article_id'] = trans['article_id'].astype('int32')
trans['week'] = trans['week'].astype('int8')
trans['sales_channel_id'] = trans['sales_channel_id'].astype('category') #why Cat?

CPU times: user 2.65 s, sys: 1.76 s, total: 4.41 s
Wall time: 27.9 s


In [ ]:
#reduce to 5% for trial runs
customers = customers.sample(frac=0.05, random_state=42)
custlist = set(customers.customer_id.unique().values_host)
trans = trans.loc[trans.customer_id.isin(custlist)]
articleset = set(trans.article_id.values_host)
articles = articles.loc[articles.article_id.isin(articleset)]

In [3]:
#for final submit only
#can do for loop with one row
import pickle
with open(path_reduce/'customer_id_to_idx_dict.pkl', 'rb') as handle:
    customer_id_to_idx_dict = pickle.load(handle)
with open(path_reduce/'idx_to_customer_id_dict.pkl', 'rb') as handle:
    idx_to_customer_id_dict = pickle.load(handle)
with open(path_reduce/'article_id_to_idx_dict.pkl', 'rb') as handle:
    article_id_to_idx_dict = pickle.load(handle) 
with open(path_reduce/'idx_to_article_id_dict.pkl', 'rb') as handle:
    idx_to_article_id_dict = pickle.load(handle)

In [ ]:
# Mapping is usually the best but also int in a few hundreds can be better than category so it can still be optimized but based on the extreme (300 uniques) 
# difference is not so big so currently i dont think need to optimize column by column, here is the fast wat to compare solutions:

#Columns data information:
# for i in articles.columns:
#     print(f'{i} has {articles[i].nunique()} and {articles[i].dtype}  ')

#mapping:
# col='department_name'
# col_unique = articles[col].unique()
# Dicti={}
# Dicti[f'{col}_to_idx_dict'] = dict(zip(col_unique.to_pandas(), col_unique.index))
# Dicti[f'idx_to_{col}_dict'] = dict(zip(col_unique.index,col_unique.to_pandas()))
# typ = DataHelper()._min_int_for_dict(articles[col].nunique())
# articles['dnmmap'] = articles[col].map(Dicti[f'{col}_to_idx_dict']).astype(typ)

#categize:
# articles['dnmcat'] = pd.Categorical(articles['department_name'].to_pandas())

#intigerize:
# articles['dnmint'] = articles['department_name'].astype('int16')
#can reduce also floar in dont need high accuracy like done in price

#---------------------------------------------------------------------------------------------------

#to speed up its better to save all data (dicts and tables) after the reducion
#here is all the commands, currenlty when reloading the table the type is not the same
#all total time is 2 minutes can investigate is more in the future, here are the commands:
# Dicts:
# import pickle

# def save_object(obj, filename):
#     with open(filename, 'wb') as outp:  # Overwrites any existing file.
#         pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

# save_object(DHC, 'customers_object.pkl')

# with open('../input/hmcompsolo/customers_object.pkl', 'rb') as inp:
#     cusobj = pickle.load(inp)
#tables:
# customers.to_pandas().to_parquet('cus.pqt') or ('customers.csv') #parquet currently not support Category type


# Adding features

In [ ]:
## Articles features

#adding 2-3 features for items:
#Mean priceVVVVVVVVVVVVVVVVVVVVVVV
#discountVVVVVVVVVVVVVVVVVVVVVVVVV
#top K sales? VVVVVVVVVVVVVVVVVVVVVVVVVVV
#Season by name ('coat', short pants, swim suit...) VVVVVVVVVVVV
#mean sales per week
#Changing class LastDiscount to only cudf (has.to_pandas( ) makes it very slow)
#adding some tqdm and status pring when extractions and collection features
review this grouping by age/sex https://www.kaggle.com/code/junjitakeshima/h-m-easy-grouping-by-sex-attribute-age-en-jp
review this for addtional features: https://www.kaggle.com/code/iwatatakuya/22nd-place-lgbm-model-single-train

In [4]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Union

class ItemFeatures(ABC):
    """ calculate features to articles."""
    @abstractmethod
    def get(self, *args, **kwargs) -> pd.DataFrame: #change to cudf
        """ article_id -> features"""
        pass

class SalesCount(ItemFeatures):
    """
    count number of transaction  
    """
    
    def __init__(self, trans: cudf.DataFrame):
        self.df = trans
        pass
    
    def get(self):
        sell_count = (self.df.groupby('article_id')
            .price
            .count()
            .to_frame()
            .rename(columns  = {'price':'sell_count'})
        )
        return sell_count

class MeanAge(ItemFeatures):
    """
    mean age of customers per article  
    """
    
    def __init__(self, trans: cudf.DataFrame, customers: cudf.DataFrame):
        self.df = trans.merge(customers[['customer_id', 'age']])
        self.df['age'] = self.df.age.astype('int8')
        pass
    
    def get(self):
        mean_age = (self.df.groupby('article_id')
            .age
            .mean()
            .astype('float32')
            .to_frame()
            .rename(columns  = {'age':'mean_age'})
        )
        return mean_age

class MeanPrice(ItemFeatures):
    """
    mean price of item transaction aggrigation 
    """
    
    def __init__(self, trans: cudf.DataFrame):
        self.df = trans
        pass
    
    def get(self):
        mean_price = (self.df.groupby('article_id')
            .price
            .mean()
            .astype('float32')
            .to_frame()
            .rename(columns  = {'price':'mean_price'})
        )
        return mean_price
    

class LastDiscount(ItemFeatures):
    """
    calcultes items last transaction price vs. highest price
    """
    
    def __init__(self, trans: cudf.DataFrame):
        self.df = trans
        pass

    
    def get(self):
        last_discount = (self.df.to_pandas()
          .assign(max1=lambda x: x.groupby('article_id')['price'].transform(max))
          .assign(last_price =lambda x: x.sort_values(by='t_dat', ascending=False )
                  .groupby('article_id').price
                  .transform(lambda z: z.iloc[0]) #sort first to verity lat date
                 )
          .assign(last_discount=lambda x: x['last_price']/x['max1'])
          .pipe(lambda x: cudf.from_pandas(x))
          .pipe(lambda x: x[['article_id', 'last_discount']])
          .drop_duplicates()
          .set_index('article_id')
        )
        return last_discount   
    
    
class ItemSeason(ItemFeatures):
    """
    add summer/winter classification based on key words in the description 
    """
    
    def __init__(self, articles: cudf.DataFrame, mapdict: Dict ):
        self.df = articles[['article_id', 'detail_desc']].copy()
        self.df['detail_desc'] = (
            self.df['detail_desc']
            .map(mapdict['idx_to_detail_desc_dict'])
        )
        pass
    
    
    def get(self):
        seasons = self.df.assign(season_type = 0)
        seasons.loc[seasons.to_pandas().detail_desc.map(self._is_winter, na_action='ignore'), "season_type"] = 1
        seasons.loc[seasons.to_pandas().detail_desc.map(self._is_summer, na_action='ignore'), "season_type"] = 2
        seasons = seasons.set_index('article_id')
        seasons = seasons.drop(columns = ['detail_desc'])
        return seasons
    
    def _is_winter(self, st: str):
        winter = {
         'beanie',
         'boots',
         'cardigan',
         'coat',
         'earmuffs',
         'fleece',
         'gloves',
         'hoodie',
         'jacket',
         'knitted',
         'long-sleeve',
         'mittens',
         'overcoat',
         'pajamas',
         'scarf',
         'ski',
         'sweater',
         'winter'
        }
        desc = set(st[:-1].lower().split())
        intersct = len(winter.intersection(desc))
        return (intersct !=0)
    
    def _is_summer(self, st):
        summer = {
         'bikini',
         'cup',
         'flip',
         'flops',
         'hawaiian',
         'sandals',
         'sleeveless',
         'summer',
         'sunglasses',
         'swim',
         'swimsuit',
         'tank',
         'tanktop'
        }
        desc = set(st[:-1].lower().split())
        intersct = len(summer.intersection(desc))
        return (intersct !=0)
# seasons = ItemSeason(articles, DHA.mapdict)

class TopK(ItemFeatures):
    """
    is article in the top k sales
    """
    
    def __init__(self, trans: cudf.DataFrame, topk: int):
        self.df = trans[['article_id']]
        self.k  = topk
        pass
    
    def get(self):
        topk = self._top_k_articles()
        is_top_k = self.df.assign(is_top = 0)
        is_top_k.loc[is_top_k.article_id.isin(topk), 'is_top'] = 1
        is_top_k = (
            is_top_k.rename(columns  = {'is_top': f'is_top_{self.k}'})
            .drop_duplicates()
            .set_index('article_id')
        )
        return is_top_k
    
    def _top_k_articles(self):
        topk= (
            self.df
            .article_id
            .value_counts()
            .nlargest(self.k)
            .index
            .to_arrow())
        return topk



class FeaturesCollect():
    """add all requested features to the dataframe."""  
    def __init__(self,df: cudf.DataFrame, features: List):
        self.df  = df
        self.features  = features
        pass
    
    def collect(self):
        df_and_features = self.df.copy()
        for ft in self.features:
            ftdf = ft.get()
            df_and_features = df_and_features.merge(ftdf, left_on='article_id', right_index=True, how='left' )
        return df_and_features        

In [5]:
%%time
# ft = [TopK(trans, 3), TopK(trans, 5), MeanPrice(trans), ItemSeason(articles, DHA.mapdict)] #LastDiscount(trans)]
ft = [SalesCount(trans), MeanPrice(trans), MeanAge(trans, customers)]
fcl =FeaturesCollect(articles, ft)
articles2f = fcl.collect()
del articles
del customers['age'] #move to before merge

CPU times: user 181 ms, sys: 72.8 ms, total: 254 ms
Wall time: 265 ms


## Customers features

In [ ]:
#Same idea as customers, probably can use same collector 
#need user class or use the class of items?
#https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/324185 - ךaddtional ideas
#also have there Customer-item feature!!!

#adding customer features:
#'cheap' buyer - buys only discounts
#gender
#purchase stats (sum, mean.. etc..)
#re-purchase level

# Setting CV

## score metric map@12

In [ ]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# Generate candidates

## Retrieval rules

In [6]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Union

class RetrivalRules(ABC):
    """ calculte and retrive candidates base on specific rule."""
    @abstractmethod
    def retrieve(self, *args, **kwargs) -> cudf.DataFrame: 
        """df->candidates"""  
        pass

class CustomerHistory2(RetrivalRules):
    """Generate candidates based on the customer transaction History."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.name = f'c_history_{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        self.weekdict = {}
        df = trans.loc[trans.week>valid_week, ['customer_id', 'article_id', 'week']]
        for w in range(valid_week+1, n_weeks+valid_week+1):
            self.weekdict[f'w{w}'] = df.loc[df.week<=w]
        pass
    
    def retrieve(self, cusotmer_list): 
        week_history_dict = self._weekly_sales_dict() 
        cust_best = {}
        for c in cusotmer_list:
            for w in week_history_dict:
                if c in week_history_dict[w]:
                    tmp = week_history_dict[w][c]
                    c_top = sorted(tmp.items(), key=lambda item: -item[1])
                    cust_best[c] = c_top[:self.n_articles]
                    break
        k = cust_best.keys() #customers
        v = cust_best.values() #tuple list (item, score)
        mut = pd.DataFrame({'customer_id': k, 'article_id': v}).explode('article_id')
        #splits the tuple value into two separtate columns in the fastest way
        mut = mut.assign(**dict(zip('ef', zip(*mut.article_id)))).drop(columns=['article_id']).rename(columns={'e':'article_id', 'f':'score'})
        mut['article_id'] = mut['article_id'].astype('int32')
        pairs = cudf.from_pandas(mut) 
        pairs[self.name] = np.int8(1)
        pairs['score'] = pairs['score'].astype('int16') #-----added to reduce memory
        pairs = pairs.rename(columns = {'score': f'score_{self.name}'})
        return pairs
    
    def _weekly_sales_dict(self):
        week_history_dict = {}
        for w, t in self.weekdict.items():
            week_history_dict[w] = {}
            for c, a in zip(t.customer_id.values_host, t.article_id.values_host):
                if c not in week_history_dict[w]:
                    week_history_dict[w][c] = {}
                if a not in week_history_dict[w][c]:
                        week_history_dict[w][c][a] = 0
                week_history_dict[w][c][a] += 1
        return week_history_dict

#in order not to build the pairs twice (neg and candidates) need update that it load file if we flag it
class basket_analysis(RetrivalRules):
    """return duct of item number and np.array of basket"""
    
    def __init__(self, trans, basket_rank=0, load=True):
        self.name = f'basket_rank{basket_rank}'
        self.t = (trans[['customer_id', 'article_id']].
                  drop_duplicates()
                 )
        self.basket_rank = basket_rank
        self.load = load
        pass

    def retrieve(self, candidates, t3):
        if self.load:
            final_pairs = t3
        else:
            final_pairs = self._basket_pairs()
# need to add save file with stgy name, load file with srgy name if we flag it
        basket_cadidates = (candidates.merge(final_pairs)
                            .drop(columns = ['article_id'])
                            .rename(columns = {'pair_id':'article_id'})
                           )
        basket_cadidates = basket_cadidates.groupby(['customer_id', 'article_id']).score.sum().reset_index()
        basket_cadidates[self.name] = np.int8(1)
        basket_cadidates['score'] = basket_cadidates['score'].astype('int32') #-----added to reduce memory
        basket_cadidates = basket_cadidates.rename(columns = {'score': f'score_{self.name}'})
        return basket_cadidates
    
    def _basket_pairs(self):
        one_timer = (self.t.groupby('article_id')[['customer_id']]
             .nunique()
             .query('customer_id==1')
             .index
        )
        to_drop = self.t[self.t.article_id.isin(one_timer)].index
        t=self.t.drop(to_drop)
        copy_pair = t.copy()
        copy_pair.columns = ['customer_id', 'pair_id'] #renaming
        unique_articles = t.article_id.unique()
        
        batch_size = 5000
        pairs_list = []
        
        for i in range(0, len(unique_articles), batch_size):
            article_batch = unique_articles[i : i+batch_size]
            pairs = t[t.article_id.isin(article_batch)]
            pairs = pairs.merge(copy_pair, on=['customer_id'])
            #remove identical pairs
            drop_idx = pairs.loc[pairs.article_id==pairs.pair_id].index
            pairs = pairs.drop(drop_idx)
            #remove cases of other customers bought just this item
            c1s = (pairs.groupby("article_id")[["customer_id"]]
                .nunique()
                .query("customer_id==1")
                .index
                  )
            single_buy_row_idxs = pairs[pairs["article_id"].isin(c1s)].index
            pairs = pairs.drop(single_buy_row_idxs)
            #count top pairs
            pairs = (pairs.groupby(['article_id', 'pair_id'])
                     .customer_id
                     .count()
                     .reset_index()
                     .rename(columns={'customer_id':'pair_count'})
                     .sort_values(by=['article_id', 'pair_count'], ascending = False)
                    )
            pairs = pairs[pairs.groupby('article_id').cumcount()==self.basket_rank] #chnge the 0 to if need 2nd one 
            pairs_list.append(pairs)
        finalpairs = cudf.concat(pairs_list)
            #normalizing the score
#         scaler = MinMaxScaler()
#         finalpairs['pair_count'] = scaler.fit_transform(finalpairs.pair_count.values_host.reshape(-1, 1)).flatten()
        finalpairs = finalpairs.rename(columns = {'pair_count':'score'})
        return finalpairs

class TopSales2(RetrivalRules):
    """Generate candidates based on sales quantity."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.name = f'top_sales_{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        self.df = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)), ['article_id', 'week']]
        pass

    def retrieve(self, customer_list):
        top_series = self.df.article_id.value_counts()[:self.n_articles]
        top_articles = top_series.index
        top_scores = top_series.values
        customers = np.repeat(customer_list, self.n_articles)
        articles = np.tile(top_articles, len(customer_list))
        scores = np.tile(top_scores, len(customer_list))
        top_sells = cudf.DataFrame({'customer_id':customers,
                                    'article_id': articles,
                                    f'score_{self.name}':scores})
        top_sells[self.name] = np.int8(1)
        return top_sells

class TopAgeGroup(RetrivalRules):
    """Generate candidtes based on top sales per agegroup"""
    def __init__(self, trans, customers, n_weeks=4, n_articles=12, valid_week=0):
        self.name = f'top_age_group{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        customers_age_group = customers[['customer_id', 'age_group']]
        customers_age_group = customers_age_group.dropna(subset=['age_group'])
        self.customers_age_group = customers_age_group
        self.df = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)),['t_dat', 'customer_id', 'article_id']]
        self.df = cudf.merge(self.df, customers_age_group, on='customer_id')
        pass

    def retrieve(self, customer_list):
        top_age = self._top_age()
        res = pd.DataFrame({'customer_id': customer_list})
        res = res.merge(self.customers_age_group.to_pandas(), on='customer_id')
        res = res.merge(top_age, on='age_group', how='left' )
        res = res.explode('scored_article')
        res = res.assign(**dict(zip(['article_id', 'score'], zip(*res.scored_article)))).drop(columns=['age_group', 'scored_article'])
        #https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
        res['article_id'] = res['article_id'].astype('int32')
        res = cudf.from_pandas(res)
        res[self.name] = np.int8(1)
        res['score'] = res['score'].astype('int32')
        res = res.rename(columns = {'score': f'score_{self.name}'})
        return res
    
    def _top_age(self):
        top_age = self.df.groupby(['age_group', 'article_id'])['t_dat'].count().reset_index().rename(columns = {'t_dat': f'score_age'})
        top_age = top_age.sort_values(by=['age_group', 'score_age' ], ascending=[True, False])
        top_age = top_age.to_pandas()
        top_age = top_age.groupby('age_group').head(self.n_articles)
        top_age['scored_article'] = list(zip(top_age.article_id, top_age.score_age))
        top_age = top_age.groupby('age_group').scored_article.apply(list).reset_index()#.rename(columns = {'article_id': 'top_age'})
        return top_age

In [ ]:
#rules not in use
class TopTrend(RetrivalRules):
    """Generate candidates based on sales trends."""
    def __init__(self, trans, n_weeks=-1, n_articles=12, valid_week=0):
        self.name = f'top_trend_{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        self.v_week = valid_week
        if n_weeks != -1:
            self.df = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)), ['article_id', 'week', 't_dat']]
        else:
            self.df = trans.loc[(trans.week.values>valid_week), ['article_id', 'week', 't_dat']]
        pass

    def retrieve(self, customer_list):
        trends = self._add_trend()
        top_trends = trends.groupby('article_id')['quotient'].sum()
        top_trends = top_trends.nlargest(self.n_articles).astype('int32')
        top_articles = top_trends.index
        top_scores = top_trends.values
        customers = np.repeat(customer_list, self.n_articles)
        articles = np.tile(top_articles, len(customer_list))
        scores = np.tile(top_scores, len(customer_list))
        res = cudf.DataFrame({'customer_id':customers,
                                    'article_id': articles,
                                    f'score_{self.name}':scores})
        res[self.name] = np.int8(1)
        return res
    
    def _add_trend(self):
        weekly_sales = self.df.groupby(['week', 'article_id']).count()
        weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
        trend_df = self.df.merge(weekly_sales, on=['week', 'article_id'])
        weekly_sales = weekly_sales.reset_index().set_index('article_id')
        trend_df = trend_df.merge(
            weekly_sales.loc[weekly_sales['week']==self.v_week+1, ['count']],
            on='article_id', suffixes=("", "_targ"), how='left')
        trend_df['count_targ'] = trend_df['count_targ'].fillna(0)
        trend_df['quotient'] = trend_df['count_targ'] / trend_df['count']
        return trend_df

class CustomerTrendHistory(RetrivalRules):
    """Generate candidates based on sales trends."""
    def __init__(self, trans, n_weeks=-1, n_articles=12, valid_week=0):
        self.name = f'c_trend_history_{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        self.v_week = valid_week
        if n_weeks != -1:
            self.df = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)), ['article_id','customer_id', 'week', 't_dat']]
        else:
            self.df = trans.loc[(trans.week.values>valid_week), ['article_id','customer_id', 'week', 't_dat']]
        pass

    def retrieve(self, customer_list):
        df2 = self._add_trend()
        last_ts = df2['t_dat'].max()
        tmp = df2.copy().to_pandas()
        tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
        tmp['dummy_1'] = 1 
        tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1) #replace 0 with 1.3 maybe faster

        a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
        tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c*tmp['x']) - d

        tmp['dummy_0'] = 0 
        tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1) #replace below zero with zero
        tmp['value'] = tmp['quotient'] * tmp['y'] 
#change value to self.nme
        tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
        tmp = tmp.reset_index()
        tmp['value'] = tmp['value'].astype('int32')
        # val = val.query('value>100') must if we use also history top sales
        tmp = tmp.sort_values(by=['customer_id', 'value'], ascending = False)
        tmp = tmp.groupby('customer_id').head(self.n_articles)
        tmp = cudf.from_pandas(tmp)
        tmp = tmp.loc[tmp.customer_id.isin(customer_list)] #move up to save time(clac only in relevant cust)
        tmp[self.name] = np.int8(1)
        a = self.name
#         tmp = tmp.query('value>0')
        tmp = tmp.rename(columns = {'value': f'score_{a}'})

        return tmp
       
    
    def _add_trend(self):
       
        weekly_sales = self.df.drop('customer_id', axis=1).groupby(['week', 'article_id']).count() #----
        weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
        
        trend_df = self.df.merge(weekly_sales, on=['week', 'article_id'])
        weekly_sales = weekly_sales.reset_index().set_index('article_id')
        trend_df = trend_df.merge(
            weekly_sales.loc[weekly_sales['week']==self.v_week+1, ['count']],
            on='article_id', suffixes=("", "_targ"), how='left')
        trend_df['count_targ'] = trend_df['count_targ'].fillna(0)
        trend_df['quotient'] = trend_df['count_targ'] / trend_df['count']
        return trend_df

In [ ]:
#classes old versions history
#customer history rule vesions history
#original first version
class CustomerHistory2(RetrivalRules):
    """Generate candidates based on the customer transaction History."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.n_articles = n_articles
        self.valid_week = valid_week
        df = trans.loc[trans.week>valid_week, ['customer_id', 'article_id', 'week']]
        self.weekdict = {}
        for w in range(valid_week+1, n_weeks+valid_week+1):
            self.weekdict[f'w{w}'] = df.loc[df.week<=w]
        pass
    
    def retrieve(self, cusotmer_list):
        week_history_dict, week_dummy_dict = self._weekly_sales_dict()
        cust_best = {}
        for c in cusotmer_list:
            for w in week_history_dict:
                if c in week_history_dict[w]:
                    tmp = week_history_dict[w][c]
                    c_top = list((dict(sorted(tmp.items(), key=lambda item: -item[1]))).keys())
                    if len(c_top) > self.n_articles:
                        cust_best[c] = c_top[:self.n_articles]
                    else:
                        cust_best[c] = c_top + week_dummy_dict[w].values.tolist()[len(c_top):self.n_articles] #chage to latest week
                    break
            if c not in cust_best:
                cust_best[c] = week_dummy_dict['w'+str(self.valid_week+1)].values.tolist()[:self.n_articles]
        k = cust_best.keys()
        v = cust_best.values()
        pairs = (cudf.DataFrame({'customer_id': k, 'article_id': v})
                 .explode('article_id')
                 .astype('int32')
                )
        return pairs
    
    def _weekly_sales_dict(self):
        week_history_dict = {}
        week_dummy_dict = {}
        for w, t in self.weekdict.items():
            week_dummy_dict[w] = t.article_id.value_counts().nlargest(self.n_articles).index
            week_history_dict[w] = {}
            for c, a in zip(t.to_pandas().customer_id, t.to_pandas().article_id):
                
                if c not in week_history_dict[w]:
                    week_history_dict[w][c] = {}
                    
                if a not in week_history_dict[w][c]:
                        week_history_dict[w][c][a] = 0
                        
                week_history_dict[w][c][a] += 1
                
        return week_history_dict, week_dummy_dict

#old reduced time with top sales
class CustomerHistory(RetrivalRules):
    """Generate candidates based on the customer transaction History."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.n_articles = n_articles
        self.valid_week = valid_week
        df = trans.loc[trans.week>valid_week, ['customer_id', 'article_id', 'week']]
        df2 = df.loc[df.week<=valid_week+n_weeks].copy()
        top_n = df2.article_id.value_counts().index[:n_articles].values.tolist()
        self.top = list(zip(top_n, [0]*n_articles))
        self.weekdict = {}
        for w in range(valid_week+1, n_weeks+valid_week+1):
            self.weekdict[f'w{w}'] = df.loc[df.week<=w]
        pass
    
    def retrieve(self, cusotmer_list):
        week_history_dict = self._weekly_sales_dict() #, week_dummy_dict
        cust_best = {}
        for c in cusotmer_list:
            for w in week_history_dict:
                if c in week_history_dict[w]:
                    tmp = week_history_dict[w][c]
                    c_top = sorted(tmp.items(), key=lambda item: -item[1])
                    if len(c_top) > self.n_articles:
                        cust_best[c] = c_top[:self.n_articles]
                    else:
                        cust_best[c] = c_top + self.top[len(c_top):self.n_articles]
                    break
            if c not in cust_best:
                cust_best[c] = self.top[:self.n_articles]
        k = cust_best.keys() #kustomer
        v = cust_best.values() #tuple list
        mut = pd.DataFrame({'customer_id': k, 'article_id': v}).explode('article_id')
        mut = mut.assign(**dict(zip('ef', zip(*mut.article_id)))).drop(columns=['article_id']).rename(columns={'e':'article_id', 'f':'score'})
        mut['article_id'] = mut['article_id'].astype('int32')
        pairs = cudf.DataFrame(mut)
        return pairs
    
    def _weekly_sales_dict(self):
        week_history_dict = {}
#         week_dummy_dict = {}
        for w, t in self.weekdict.items():
#             week_dummy_dict[w] = t.article_id.value_counts().nlargest(self.n_articles).index
            week_history_dict[w] = {}
            for c, a in zip(t.to_pandas().customer_id, t.to_pandas().article_id):
                
                if c not in week_history_dict[w]:
                    week_history_dict[w][c] = {}
                    
                if a not in week_history_dict[w][c]:
                        week_history_dict[w][c][a] = 0
                        
                week_history_dict[w][c][a] += 1
                
        return week_history_dict#, week_dummy_dict

#new with fetures but slow so change to the one above
class CustomerHistory5(RetrivalRules):
    """Generate candidates based on the customer transaction History."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.n_articles = n_articles
        self.valid_week = valid_week
        self.n_weeks = n_weeks
        self.df = trans.loc[(trans.week>valid_week)&(trans.week<=(valid_week+n_weeks)), ['customer_id', 'article_id', 'week']]
        pass
    
    def retrieve(self, customer_list):
        pairs = []
        top_sales = self.df.article_id.value_counts().nlargest(self.n_articles).reset_index().rename(columns={'index':'article_id', 'article_id':'score'})
        top_sales['score']=0
        for c in customer_list:
            if c in set(self.df.customer_id.values_host):
                minweek = self.df[self.df.customer_id.values==c].week.min()
                #next slice actaully shuold be <= minweek to make the same
                top_c = self.df.loc[(self.df.week.values==minweek)&(self.df.customer_id.values==c)].article_id.value_counts().reset_index().rename(columns={'index':'article_id', 'article_id':'score'})
                top_c = cudf.concat([top_c, top_sales]).iloc[:self.n_articles]
                top_c['customer_id'] = c   
            else:
                top_c = top_sales.copy()
                top_c['customer_id'] = c
            pairs.append(top_c)
        return cudf.concat(pairs)

#possbly fastest way for this recall from the chosen solution 

%%time
yourList = list(cust_best.keys())
n = 15
len(np.repeat(yourList, n))

for a in list(dici.values()):
    d = dict(a)
#     print(d)
    print(d.values())
#     print(d.keys()) #why missing first row?
dict_values([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
dict_values([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

import itertools
list(itertools.chain.from_iterable([list(dict(a).values()) for a in dici.values()]))
# new_list = [x for x in chain.from_iterable(tupList)]

v = [(74739, 4), (57658, 4), (14253, 2), (94674, 1), (57643, 1)]
print(dict(v).keys())
print(dict(v).values())
# k = list('abcde')
#end of customer history version

#basket rule version history and score shift:

#score shift
#As the transfer score is currently much higher we will temporary  use it
#in order to create we build the regular cudf using the class, than we build again in the old was creating the dics
#then we swich score in 2 steps: 1. swich the article_id to the pair_id 2. swich score to the transform Id
#its because of a mistake in the order in old solution that actially gave a big boost
#I think that its the new higher score is for more popular items and some how predict better SO:
# good chance that populrity high selling score will do it as a feature and than we can go back to original score


# unhide if its not test or hide if its for test
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

test_week = 0
trans, test = weeksplit(trans, test_week)
#end of hide/unhide

ba = basket_analysis(trans)
t3 = ba._basket_pairs()

#new adding score
#to prevent leakgae split the test first..................
t = trans[['customer_id', 'article_id']]
t = t.drop_duplicates()
#reduce some timer to save memory/increase speed
onetimer = t.groupby('article_id')[['customer_id']].nunique().query('customer_id==1').index
to_drop = t[t.article_id.isin(onetimer)].index
t=t.drop(to_drop)

copy_pair = t.copy()
copy_pair.columns = ['customer_id', 'pair_id']
unique_articles = t.article_id.unique()

batch_size = 5000
pairs_list = []

for i in range(0, len(unique_articles), batch_size):
    article_batch = unique_articles[i : i+batch_size]
    pairs = t[t.article_id.isin(article_batch)]
    pairs = pairs.merge(copy_pair, on=['customer_id'])
    #remove identical pairs
    drop_idx = pairs.loc[pairs.article_id==pairs.pair_id].index
    pairs = pairs.drop(drop_idx)
#     remove cases of other customers bought just this item
    c1s = (
        pairs.groupby("article_id")[["customer_id"]].nunique()
        .query("customer_id==1").index
    )
    single_customer_row_idxs = pairs[pairs["article_id"].isin(c1s)].index
    pairs = pairs.drop(single_customer_row_idxs)
    pairs = pairs.groupby(['article_id', 'pair_id']).customer_id.count()
    pairs = pairs.reset_index().rename(columns={'customer_id':'pair_count'})
    pairs = pairs.sort_values(by=['article_id', 'pair_count'], ascending = False)
    pairs = pairs[pairs.groupby('article_id').cumcount()==0] #chnge the 0 to if need 2nd one , verify if shorter than 5
    pairs_list.append(pairs)

finalpairs = cudf.concat(pairs_list)

#old with dicts
finalpairs = finalpairs.set_index('article_id')
new_pairs = finalpairs["pair_id"].to_pandas().to_dict()
#normlize
scaler = MinMaxScaler()
finalpairs['pair_count'] = scaler.fit_transform(finalpairs.pair_count.values_host.reshape(-1, 1)).flatten() #was 'a'??? try 'change to pair count later'
new_score = finalpairs["pair_count"].to_pandas().to_dict()

t3['trans_score'] = t3.pair_id.map(new_score) #running old first
t3 = t3.drop(columns=['score']).rename(columns = {'trans_score':'score'})
# t3.to_parquet('t3cv.pqt')

#old original version speed up of weeksplit
t = trans[['customer_id', 'article_id']]
t = t.drop_duplicates()
#reduce some timer to save memory/increase speed
onetimer = t.groupby('article_id')[['customer_id']].nunique().query('customer_id==1').index
to_drop = t[t.article_id.isin(onetimer)].index
t=t.drop(to_drop)

copy_pair = t.copy()
copy_pair.columns = ['customer_id', 'pair_id']
unique_articles = t.article_id.unique()

batch_size = 5000
pairs_list = []

for i in range(0, len(unique_articles), batch_size):
    article_batch = unique_articles[i : i+batch_size]
    pairs = t[t.article_id.isin(article_batch)]
    pairs = pairs.merge(copy_pair, on=['customer_id'])
    #remove identical pairs
    drop_idx = pairs.query('article_id==pair_id').index
    pairs = pairs.drop(drop_idx)
#     remove one timer to increase speed/save memory
    c1s = (
        pairs.groupby("article_id")[["customer_id"]].nunique()
        .query("customer_id==1").index
    )
    single_customer_row_idxs = pairs[pairs["article_id"].isin(c1s)].index
    pairs = pairs.drop(single_customer_row_idxs)

    pairs = pairs.groupby(['article_id', 'pair_id']).customer_id.count()
    pairs = pairs.reset_index().rename(columns={'customer_id':'pair_count'})
    pairs = pairs.sort_values(by=['article_id', 'pair_count'], ascending = False)
#we want the score!!! we also want stats on pair caunt (avg, count,,.)
    pairs = pairs.to_pandas().groupby('article_id').head(1) 
    pairs = cudf.DataFrame(pairs.set_index('article_id')[['pair_id']])
    pairs = cudf.DataFrame(pairs)
    pairs_list.append(pairs)

finalpairs = cudf.concat(pairs_list)

# pd.DataFrame({'A':[1,2,3],'B':[4,4,4] })[4:5]

#old original score with strgy func
scaler = MinMaxScaler()
finalpairs['pair_count'] = scaler.fit_transform(finalpairs.pair_count.values_host.reshape(-1, 1)).flatten()
def srtgy2(s, finalpairs):
    a = s.merge(finalpairs).drop(columns = ['article_id']).rename(columns = {'pair_id':'article_id', 'pair_count':'score'})
    return a
#end of basket history

#old topsales version history
class TopSales(RetrivalRules):
    """Generate candidates based on sales quantity."""
    def __init__(self, trans, n_weeks=4, n_articles=12, valid_week=0):
        self.n_articles = n_articles
        self.df = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)), ['article_id', 'week']]
        pass

    def retrieve(self, customer_list):
        #add score - one opton list of tuples with score and explode like history
        top_list = self.df.article_id.value_counts().index.values[:self.n_articles].tolist()
        top_sells = cudf.DataFrame()
        top_sells['customer_id'] = customer_list
        top_sells['article_id'] = [top_list for x in range(len(customer_list))]
        top_sells = top_sells.explode('article_id')
        return top_sells
#end of topsales history


#topagegroup version history--------------------------------------------------
class TopAgeGroup3(RetrivalRules):
    """Generate ........"""
    def __init__(self, trans, customers, n_weeks=4, n_articles=12, valid_week=0):
        customers = customers.dropna(subset=['age_group'])
        self.customers_age_group = customers[['customer_id', 'age_group']]
        self.name = f'top_age_group{n_weeks}w_{n_articles}a'
        self.n_articles = n_articles
        agegroup = trans.loc[(trans.week.values>valid_week)&(trans.week.values<(valid_week+n_weeks+1)),['t_dat', 'customer_id', 'article_id']]
        agegroup = cudf.merge(agegroup, customers[['customer_id', 'age_group']], on='customer_id')
        #sub_func
        ag = agegroup.groupby(['age_group', 'article_id'])['t_dat'].count().reset_index().rename(columns = {'t_dat': f'score_age'})
        ag = ag.sort_values(by=['age_group', 'score_age' ], ascending=[True, False])
        ag = ag.to_pandas()
        ag = ag.groupby('age_group').head(n_articles)
        ag['scored_article'] = list(zip(ag.article_id, ag.score_age))
        self.df = ag.groupby('age_group').scored_article.apply(list).reset_index().rename(columns = {'article_id': 'top_age'})
        pass

    def retrieve(self, customer_list):
        c = pd.DataFrame({'customer_id': customer_list})
        c = c.merge(self.customers_age_group.to_pandas(), on='customer_id')
        c = c.merge(self.df, on='age_group', how='left' )
        c = c.explode('scored_article')
        c = c.assign(**dict(zip(['article_id', 'score'], zip(*c.scored_article)))).drop(columns=['age_group', 'scored_article'])
        #https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
        c['article_id'] = c['article_id'].astype('int32')
        res = cudf.from_pandas(c)
        res[self.name] = np.int8(1)
        a = self.name
        res = res.rename(columns = {'score': f'score_{a}'})
        return res
# end o#topagegroup version history--------------------------------------------------

In [ ]:
#unit test for class changes
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

valid_week = 0
train, valid = weeksplit(trans, valid_week)
valid_customers = valid.customer_id.unique().values_host
# CHrate = CustomerHistory2(trans, 4, 15, valid_week)
# recalls1 = CHrate.retrieve(valid_customers)
tagc = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=valid_week)
recalls5 = tagc.retrieve(valid_customers)

tagc3 = TopAgeGroup3(trans, customers, n_weeks=1, n_articles=15, valid_week=valid_week)
recalls53 = tagc3.retrieve(valid_customers)

#check baskets
t4 = cudf.read_parquet(path_reduce/'t0sub.pqt')
t5 = cudf.read_parquet(path_reduce/'t1sub.pqt') 

BA12 = basket_analysis(trans) #send all data include all weeks except test
recalls22 = BA12.retrieve(recalls1[['customer_id','article_id']], t4) 

In [ ]:
#rule checker
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

def rule_score(actual, predict):
    actual = actual[['customer_id', 'article_id']].drop_duplicates()
    act_tot = len(actual)
    pre_tot = len(predict)
    correct = actual.merge(predict, on=['customer_id', 'article_id'], how='inner').shape[0]
    print(f"[+] Recall = {correct/act_tot*100:.4f}% ({correct}/{act_tot})")
    print(f"[+] Precision = {correct/pre_tot*100:.4f}% ({correct}/{pre_tot})")
    return f"{correct/act_tot*100:.4f}%",  f'({correct}/{act_tot})', f"{correct/pre_tot*100:.4f}%",  f'({correct}/{pre_tot})'

#old recall
# def recall_ratio(recalls, valid):
#     "get recalls and valid - returs the postive rate"
#     buys = valid[['customer_id', 'article_id']].drop_duplicates()
#     buys['label']=1
#     pos_rate = recalls.merge(buys, on=['customer_id', 'article_id'], how='left').label.fillna(0).mean()
#     return pos_rate

# recall_ratio(recalls, valid) #0.0089 (w0), 0.0078(w1), 0.0077(w2), 0.0068(w3) - min_pos_rate=0.006

t4 = cudf.read_parquet(path_reduce/'t0sub.pqt')
t5 = cudf.read_parquet(path_reduce/'t1sub.pqt') 
# t4 = t4.loc[t4.score>10]
# t4_high_score.sort_values(by=['pair_id', 'article_id'])[:30]

%%time
recall_list = []; recall_ver_l = []; precision_list = []; precision_ver_list = []
for valid_week in [0, 1, 2]:#, 1, 2]:#, 3, 4, 5, 6, 7, 8, 9]:
    print(f'this is for valid week{valid_week}')
    train, valid = weeksplit(trans, valid_week)
    valid_customers = valid.customer_id.unique().values_host
    CHrate = CustomerHistory2(trans, 4, 15, valid_week)
    recalls1 = CHrate.retrieve(valid_customers)
    
#     cth = CustomerTrendHistory(trans, n_weeks=4, n_articles=15, valid_week=valid_week)
#     recalls10 = cth.retrieve(valid_customers)
    
    
    
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    recalls2 = BA1.retrieve(recalls1[['customer_id','article_id']], t4) 
    
#     recalls2 = recalls2.drop_duplicates(['customer_id', 'article_id'])
    recalls4 = BA1.retrieve(recalls1[['customer_id','article_id']], t5)
     

    ts = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    recalls3 = ts.retrieve(valid_customers)
    
    ts2 = TopSales2(trans, n_weeks=10, n_articles=10, valid_week=valid_week)
    recalls7 = ts2.retrieve(valid_customers)

    tagc = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=valid_week)
    recalls5 = tagc.retrieve(valid_customers)
    
#     tpf = TopTrend(trans, n_weeks=-1, n_articles=10, valid_week=valid_week)
#     recalls6 = tpf.retrieve(valid_customers)
   
    recallsmrg = recalls1.copy()
    for rec3 in [recalls2, recalls3, recalls4, recalls5, recalls7]:
        before_merge = recallsmrg.shape[0] #to delete
        union = recallsmrg.merge(rec3, on=['customer_id', 'article_id'], how='inner')
        recallsmrg = recallsmrg.merge(rec3, on=['customer_id', 'article_id'], how='outer')
        x = recallsmrg.shape[0] - before_merge - rec3.shape[0] + union.shape[0]
        print(f'no duplicate verification {x}')
    print(f'total merge {recallsmrg.shape}')
    
    rec, rec_v, pre, pre_v = rule_score(valid, recallsmrg)
    recall_list.append(rec); recall_ver_l.append(rec_v); precision_list.append(pre); precision_ver_list.append(pre_v)
    print('\n')

pd.DataFrame([recall_list, recall_ver_l, precision_list, precision_ver_list])


## Intenal CV

In [ ]:
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

test_week = 0
trans, test = weeksplit(trans, test_week)

In [ ]:
# * WEEK_NUM = 0: test
# * WEEK_NUM = 1, 2: valid
# * WEEK_NUM > 1, 2: train
#split test and train
#split again to train and valid

#old with 2nd recall stg and score + normilize
recalls = []
for valid_week in [1, 2, 3]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH1 = CustomerHistory(trans, 4, 15, valid_week)
    negdf1 = CH1.retrieve(custlist)
    negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten()
    negdf1['strtgy1'] = np.int8(1)
    negdf2 = srtgy2(negdf1.query('score>0')[['customer_id','article_id']], finalpairs)
    
#     negdf2 = negdf1.copy()
#     negdf2 = negdf2.loc[negdf2.score.values>0]     
#     negdf2['score'] = negdf2.article_id.map(new_score)
#     negdf2['article_id'] = negdf2.article_id.map(new_pairs)
    negdf2['strtgy2'] = np.int8(1)
# #     negdf2['strtgy1'] = np.int8(0)
# #     negdf2 = negdf2.drop_duplicates()
    negdf = cudf.concat([negdf1, negdf2]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0}) #change to inner or left
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)
   


#old with 2nd recall stg and score + normilize + class
recalls = []
for valid_week in [1, 2, 3]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH1 = CustomerHistory(trans, 4, 15, valid_week)
    negdf1 = CH1.retrieve(custlist)
    scaler = MinMaxScaler()
    negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten() #move to class
    negdf1['strtgy1'] = np.int8(1)
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1.query('score>0')[['customer_id','article_id']], t3)
#     negdf2 = srtgy2(negdf1.query('score>0')[['customer_id','article_id']], finalpairs)
#     negdf2 = negdf1.copy()
#     negdf2 = negdf2.loc[negdf2.score.values>0]     
#     negdf2['score'] = negdf2.article_id.map(new_score)
#     negdf2['article_id'] = negdf2.article_id.map(new_pairs)
    negdf2['strtgy2'] = np.int8(1)
# #     negdf2['strtgy1'] = np.int8(0)
# #     negdf2 = negdf2.drop_duplicates()
    negdf = cudf.concat([negdf1, negdf2]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0}) #change to inner or left
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

#old with 2nd history and top split
recalls = []
for valid_week in [1, 2, 3]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH1 = CustomerHistory2(trans, 4, 15, valid_week) #ch to 2
    negdf1 = CH1.retrieve(custlist)
    scaler = MinMaxScaler()
#     negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten() #move to class
    #     negdf1 = negdf1.rename(columns={"score": "st1_score"})
    negdf1['strtgy1'] = np.int8(1)
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1[['customer_id','article_id']], t3) #didnt filter the zero
    negdf2['strtgy2'] = np.int8(1)
    ts = TopSales(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    negdf3 = ts.retrieve(custlist)
    negdf3['strtgy3'] = np.int8(1)
    negdf3['score'] = np.int8(0)
    #     negdf3['st3_score'] = np.int8(0)

    negdf = cudf.concat([negdf1, negdf2, negdf3]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0}) #change to  left
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

In [ ]:
#winner

t4 = cudf.read_parquet(path_reduce/'t0cv.pqt') #'t0sub.pqt' -for test, and t0cv.pqt-for internal cv
t5 = cudf.read_parquet(path_reduce/'t1cv.pqt') #'t1sub.pqt' -for test, and t1cv.pqt-for internal cv

recalls = []
for valid_week in [1, 2, 3]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host  
    CH1 = CustomerHistory2(trans, 4, 15, valid_week) 
    negdf1 = CH1.retrieve(custlist)

    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1[['customer_id','article_id']], t4) #didnt filter the zero
    negdf5 = BA1.retrieve(negdf1[['customer_id','article_id']], t5) 
    negdf2 = cudf.concat([negdf2, negdf5])
    negdf2 = negdf2.drop_duplicates(['customer_id', 'article_id']) #check regaring the name and score
    
    ts = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    negdf3 = ts.retrieve(custlist)
    
    ts2 = TopSales2(trans, n_weeks=10, n_articles=10, valid_week=valid_week)
    negdf7 = ts2.retrieve(custlist)
    
    tag = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=valid_week) 
    negdf4 = tag.retrieve(custlist)
   
    negdf = negdf1.copy()
    for neg in [negdf2, negdf3, negdf4, negdf7]:
        negdf = negdf.merge(neg, on=['customer_id', 'article_id'], how='outer')
    negdf = negdf.fillna(0)
    strgy_names = [x for x in negdf.columns if 'score' not in x][2:]
    negdf['strgy_count'] = negdf[strgy_names].sum(axis=1).astype('int8')
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='left').fillna({'label':0}) 
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

trainfin = cudf.concat(recalls) #keep
trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles2f, on='article_id') #change to aff features
)

print(trainfin.label.value_counts(dropna=False))

In [ ]:
#training lightgbm ranker final
trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),#added valid week----
    trainfin.pop('label'),
    group=train_baskets,
)

In [ ]:
#trainfin playground with eval

trainfin = trainfin.to_pandas()
trainfin_train = trainfin.loc[trainfin.valid_week<3].copy()
trainfin_valid = trainfin.loc[trainfin.valid_week==3].copy()

trainfin_train.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets_qids = trainfin_train.groupby(['valid_week', 'customer_id'])['article_id'].count().values

trainfin_valid.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets_qids_valid = trainfin_valid.groupby(['valid_week', 'customer_id'])['article_id'].count().values

from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

ranker = ranker.fit(
    X=trainfin_train.drop(columns = [ 'customer_id', 'article_id', 'label']),
    y=trainfin_train.pop('label'),
    group=train_baskets_qids,
    eval_set=[(trainfin_valid.drop(columns = [ 'customer_id', 'article_id', 'label']), trainfin_valid.pop('label'))],
    eval_group=[train_baskets_qids_valid],
    eval_at=10,
    verbose=10,
)

In [ ]:
imp = (ranker.feature_importances_ / ranker.feature_importances_.sum())*100
desc = ranker.feature_name_
pd.DataFrame({'name': desc, 'score': imp}).sort_values(by=['score'], ascending=False)
# ranker.evals_result_

In [ ]:
#old with score and week
custlist3 = test.customer_id.unique().values_host 
CHv = CustomerHistory(trans, 4, 15) #defaoult valid week is 0 so its taking only train
candidates = CHv.retrieve(custlist3)
candidates['valid_week'] = test_week

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#adjusting features
candidates =  candidates[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','score']]
#----------------------------------------------------------------------

# old 2 strgy
custlist3 = test.customer_id.unique().values_host 
CHv1 = CustomerHistory(trans, 4, 15)
# CHv2 = CustomerHistory(trans, 5, 15)#defaoult valid week is 0 so its taking only train
candidates1 = CHv1.retrieve(custlist3)
candidates1['strtgy1'] = np.int8(1)
# candidates2 = CHv2.retrieve(custlist3)
candidates2 = candidates1.copy()
candidates2 = candidates2.loc[candidates2.score.values>0]    
candidates2['article_id'] = candidates2.article_id.map(new_pairs)
candidates2['strtgy2'] = np.int8(1)


candidates = cudf.concat([candidates1, candidates2]).fillna(0)
candidates['valid_week'] = test_week

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#adjusting features
candidates =  candidates[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','score', 'strtgy1', 'strtgy2']] #added strgy
#---------------------------------------------------------------------------------------------------------

#old 2 strgy +2nd score normlize
custlist3 = test.customer_id.unique().values_host 
CHv1 = CustomerHistory(trans, 4, 15)

candidates1 = CHv1.retrieve(custlist3)
candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)
# candidates2 = CHv2.retrieve(custlist3)
# candidates2 = srtgy2(candidates1.article_id, finalpairs)
candidates2 = srtgy2(candidates1.query('score>0')[['customer_id','article_id']], finalpairs)

# candidates2 = candidates1.copy()
# candidates2 = candidates2.loc[candidates2.score.values>0]    
# candidates2['article_id'] = candidates2.article_id.map(new_pairs)
# candidates2['score'] = candidates2.article_id.map(new_score)
candidates2['strtgy2'] = np.int8(1)


candidates = cudf.concat([candidates1, candidates2]).fillna(0)
candidates['valid_week'] = test_week

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#add drop duplicate

#adjusting features
candidates =  candidates[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','score', 'strtgy1', 'strtgy2']] #added strgy
#-------------------------------------------------------------------------------------------------------

#old 2 strgy +2nd score normlize +class
#to aff features need change merge and columns namnes 
custlist3 = test.customer_id.unique().values_host 
CHv1 = CustomerHistory(trans, 4, 15)

candidates1 = CHv1.retrieve(custlist3)
candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1.query('score>0')[['customer_id','article_id']], t3)
# candidates2 = srtgy2(candidates1.query('score>0')[['customer_id','article_id']], finalpairs)
# candidates2 = candidates1.copy()
# candidates2 = candidates2.loc[candidates2.score.values>0]    
# candidates2['article_id'] = candidates2.article_id.map(new_pairs)
# candidates2['score'] = candidates2.article_id.map(new_score)
candidates2['strtgy2'] = np.int8(1)
candidates = cudf.concat([candidates1, candidates2]).fillna(0)
candidates['valid_week'] = test_week

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id')) #change to add featires
)

#add drop duplicate

#adjusting features
# candidates =  candidates[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
#        'graphical_appearance_name', 'colour_group_name',
#        'perceived_colour_value_name', 'perceived_colour_master_name',
#        'department_name', 'index_name', 'index_group_name', 'section_name',
#        'garment_group_name','score', 'strtgy1', 'strtgy2']] #change to add features

#----------------------------------------------------------------------------------------------------------------

#old 2 strgy history and topsales------ concat and adding manually score and names 
custlist3 = test.customer_id.unique().values_host 
CHv1 = CustomerHistory2(trans, 4, 15)
candidates1 = CHv1.retrieve(custlist3)
# candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t3)
candidates2['strtgy2'] = np.int8(1)

ts1 = TopSales(trans, n_weeks=1, n_articles=30)
candidates3 = ts1.retrieve(custlist3)
candidates3['strtgy3'] = np.int8(1)
candidates3['score'] = np.int8(0)


candidates = cudf.concat([candidates1, candidates2, candidates3]).fillna(0)
candidates['valid_week'] = test_week

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id')) #change to add featires
)

#add drop duplicate


In [ ]:
#winner
custlist3 = test.customer_id.unique().values_host 
CHv1 = CustomerHistory2(trans, 4, 15) #valid_week=0 by default  - change for readabilty 
candidates1 = CHv1.retrieve(custlist3)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t4)
candidates5 = BAv1.retrieve(candidates1[['customer_id','article_id']], t5) 
candidates2 = cudf.concat([candidates2, candidates5])
candidates2 = candidates2.drop_duplicates(['customer_id', 'article_id'])
 
ts1 = TopSales2(trans, n_weeks=1, n_articles=30) #valid_week=0 by default  - change for readabilty 
candidates3 = ts1.retrieve(custlist3)

ts2 = TopSales2(trans, n_weeks=10, n_articles=10) #valid_week=0 by default  - change for readabilty 
candidates7 = ts2.retrieve(custlist3)

tagv1 = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=0)
candidates4 = tagv1.retrieve(custlist3)

candidates = candidates1.copy()
for can in [candidates2,  candidates3,  candidates4, candidates7]:
    candidates = candidates.merge(can, on=['customer_id', 'article_id'], how='outer')
candidates = candidates.fillna(0)
strgy_names = [x for x in candidates.columns if 'score' not in x][2:] #prbly can cancel
candidates['strgy_count'] = candidates[strgy_names].sum(axis=1).astype('int8')
    
candidates['valid_week'] = np.int8(test_week) 

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles2f, on = ('article_id')) 
)
#add drop duplicate

In [ ]:
#winner
preds = []
batch_size = 10_000
for bucket in tqdm(range(0, len(candidates), batch_size)):
  outputs = ranker.predict(
      candidates.iloc[bucket: bucket+batch_size]
      .drop(columns = ['customer_id', 'article_id'])
      .to_pandas()
      )
  preds.append(outputs)

preds = np.concatenate(preds)
candidates['preds'] = preds
preds = candidates[['customer_id', 'article_id', 'preds']]
preds = preds.to_pandas() #check if cuold avoid
preds.sort_values(['customer_id', 'preds'], ascending=False, inplace = True) #can be done with cudf
preds2 = preds.groupby('customer_id').article_id.apply(list).reset_index() #must change pandas (not ittreble)
test = test.to_pandas() #check if cuold avoid
val = test.groupby('customer_id').article_id.apply(list).reset_index()
ss = val.merge(preds2, left_on='customer_id', right_on='customer_id')
mapk(ss['article_id_x'], ss['article_id_y'], k=12)

In [ ]:
test = cudf.from_pandas(test)

# Private leaderboard  - new

In [ ]:
#old - but cahnged a little in the middle
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

# test_week = 0
# trans, test = weeksplit(trans, test_week)

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH = CustomerHistory(trans, 4, 15, valid_week)
    negdf1 = CH.retrieve(custlist)
    scaler = MinMaxScaler()
    negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten()
    negdf1['strtgy1'] = np.int8(1)
    negdf2 = negdf1.copy()
    negdf2 = negdf2.loc[negdf2.score.values>0]     
    
    negdf2['score'] = negdf2.article_id.map(new_score) #chnged to correct score check if its really so low
    negdf2['article_id'] = negdf2.article_id.map(new_pairs)
    
#     negdf2['article_id'] = negdf2.article_id.map(new_pairs)
#     negdf2['score'] = negdf2.article_id.map(new_score)
    
    negdf2['strtgy2'] = np.int8(1)
    negdf = cudf.concat([negdf1, negdf2]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0})
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

#added valid score
trainfin = cudf.concat(recalls)
trainfin = trainfin[trainfin['score'].notna()] #only recalls no positive from weekly valid

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles, on='article_id')
)

trainfin =  trainfin[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','label', 'score', 'strtgy1', 'strtgy2']]

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

#training lightgbm ranker ---new
#added valid week sort and gruup
from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),
    trainfin.pop('label'),
    group=train_baskets,
)

#----history is combined----------------------------------------------------------------------------------------------------
#old
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

# test_week = 0
# trans, test = weeksplit(trans, test_week)

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH = CustomerHistory(trans, 4, 15, valid_week)
    negdf1 = CH.retrieve(custlist)
    scaler = MinMaxScaler()
    negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten()
    negdf1['strtgy1'] = np.int8(1)
    
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1.query('score>0')[['customer_id','article_id']], t3)
    negdf2['strtgy2'] = np.int8(1)
    
    negdf = cudf.concat([negdf1, negdf2]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0})
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

#added valid score
trainfin = cudf.concat(recalls)
trainfin = trainfin[trainfin['score'].notna()] #only recalls no positive from weekly valid

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles, on='article_id')
)

trainfin =  trainfin[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','label', 'score', 'strtgy1', 'strtgy2']]

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

#training lightgbm ranker ---new
#added valid week sort and gruup
from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),
    trainfin.pop('label'),
    group=train_baskets,
)
#-----------------------------------old 1st part

#old with 3 stgy  and all featres
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

# test_week = 0
# trans, test = weeksplit(trans, test_week)

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH = CustomerHistory2(trans, 4, 15, valid_week)
    negdf1 = CH.retrieve(custlist)
    scaler = MinMaxScaler()
#     negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten()
    negdf1['strtgy1'] = np.int8(1)
    
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1[['customer_id','article_id']], t3)
    negdf2['strtgy2'] = np.int8(1)
    
    ts = TopSales(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    negdf3 = ts.retrieve(custlist)
    negdf3['strtgy3'] = np.int8(1)
    negdf3['score'] = np.int8(0)
    
    negdf = cudf.concat([negdf1, negdf2, negdf3]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0})
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

#added valid score
trainfin = cudf.concat(recalls)
trainfin = trainfin[trainfin['score'].notna()] #only recalls no positive from weekly valid

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles, on='article_id')
)

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

#training lightgbm ranker ---new
#added valid week sort and gruup
from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),
    trainfin.pop('label'),
    group=train_baskets,
)
#end old 1

#old with 3 stgy  and seperated score and 'left' join
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

# test_week = 0
# trans, test = weeksplit(trans, test_week)

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH = CustomerHistory2(trans, 4, 15, valid_week)
    negdf1 = CH.retrieve(custlist)
    negdf1 = negdf1.rename(columns={"score": "st1_score"})
#     scaler = MinMaxScaler()
#     negdf1['score'] = scaler.fit_transform(negdf1.score.values_host.reshape(-1, 1)).flatten()
    negdf1['strtgy1'] = np.int8(1)
    
    BA1 = basket_analysis(trans) #send all data include all weeks except test
    negdf2 = BA1.retrieve(negdf1[['customer_id','article_id']], t3)
    negdf2 = negdf2.rename(columns={"score": "st2_score"})
    negdf2['strtgy2'] = np.int8(1)
    
    ts = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    negdf3 = ts.retrieve(custlist)
#     negdf3['strtgy3'] = np.int8(1)
#     negdf3['score'] = np.int8(0)
    
    negdf = cudf.concat([negdf1, negdf2, negdf3]).fillna(0)
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='left').fillna({'label':0})
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

#added valid score
trainfin = cudf.concat(recalls)
# trainfin = trainfin[trainfin['score'].notna()] #only recalls no positive from weekly valid

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles, on='article_id')
)

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

#training lightgbm ranker ---new
#added valid week sort and gruup
from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),
    trainfin.pop('label'),
    group=train_baskets,
)

In [7]:
#winner -final
t4 = cudf.read_parquet(path_reduce/'t0sub.pqt') #'t0sub.pqt' -for test, and t0cv.pqt-for internal cv
t5 = cudf.read_parquet(path_reduce/'t1sub.pqt') #'t1sub.pqt' -for test, and t1cv.pqt-for internal cv

#move it up? we have thi func in cv also
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host 
    CH = CustomerHistory2(trans, 4, 15, valid_week)
    negdf1 = CH.retrieve(custlist)

    BA1 = basket_analysis(trans) 
    negdf2 = BA1.retrieve(negdf1[['customer_id','article_id']], t4) 
    negdf5 = BA1.retrieve(negdf1[['customer_id','article_id']], t5)
    negdf2 = cudf.concat([negdf2, negdf5])
    negdf2 = negdf2.drop_duplicates(['customer_id', 'article_id'])
    
    ts = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=valid_week)
    negdf3 = ts.retrieve(custlist)
    
    ts2 = TopSales2(trans, n_weeks=10, n_articles=10, valid_week=valid_week+1) #valid week we take in ts
    negdf7 = ts2.retrieve(custlist)
    
    tag = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=valid_week)
    negdf4 = tag.retrieve(custlist) 
  
    negdf = negdf1.copy()
    for neg in [negdf2, negdf3, negdf4, negdf7]: 
        negdf = negdf.merge(neg, on=['customer_id', 'article_id'], how='outer')
    negdf = negdf.fillna(0)
    
    strgy_names = [x for x in negdf.columns if 'score' not in x][2:]
    negdf['strgy_count'] = negdf[strgy_names].sum(axis=1).astype('int8')
    
    negdf = negdf.drop_duplicates() #probably no need
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = np.int8(1)
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='left').fillna({'label':0})
    week_recall['valid_week'] = np.int8(valid_week)
    recalls.append(week_recall)

trainfin = cudf.concat(recalls)

print(trainfin.label.value_counts(dropna=False))

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles2f, on='article_id') #add features
)

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

from lightgbm.sklearn import LGBMRanker #moveup

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id','article_id', 'label']), 
    trainfin.pop('label'),
    group=train_baskets,
)

/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2600: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


0    9535587
1      52277
Name: label, dtype: int32


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.828266
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.328501
[LightGBM] [Debug] init for col-wise cost 1.000754 seconds, init for row-wise cost 4.691568 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 2.121045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 2492
[LightGBM] [Info] Number of data points in the train set: 9587864, number of used features: 34
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [De

In [ ]:
#old basket score dict--- to delete copy down
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])

#generate candidates
CH2 = CustomerHistory(trans, 4, 15, -1) #veify
custlist2 = sample_sub.cusid.unique().values_host
candidates1 = CH2.retrieve(custlist2)
candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)
candidates2 = candidates1.copy()
candidates2 = candidates2.loc[candidates2.score.values>0]    

candidates2['score'] = candidates2.article_id.map(new_score) #change to real acore check if its really taht low
candidates2['article_id'] = candidates2.article_id.map(new_pairs)


# candidates2['article_id'] = candidates2.article_id.map(new_pairs)
# candidates2['score'] = candidates2.article_id.map(new_score)

candidates2['strtgy2'] = np.int8(1)


candidates = cudf.concat([candidates1, candidates2]).fillna(0)
candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#old manual adjusting names----------------------------------------------------------------------------------
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
# sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)

#generate candidates
custlist2 = sample_sub.cusid.unique().values_host
CH2 = CustomerHistory2(trans, 4, 15, -1) #veify
candidates1 = CH2.retrieve(custlist2)
# candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t3)
candidates2['strtgy2'] = np.int8(1)

ts1 = TopSales(trans, n_weeks=1, n_articles=30, valid_week=-1)
candidates3 = ts1.retrieve(custlist2)
candidates3['strtgy3'] = np.int8(1)
candidates3['score'] = np.int8(0)

candidates = cudf.concat([candidates1, candidates2, candidates3]).fillna(0)
candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#old before adding candiate4----------------------------------------------------------------------------
#old final with seperated scores
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
# sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)

#generate candidates
custlist2 = sample_sub.cusid.unique().values_host
CH2 = CustomerHistory2(trans, 4, 15, -1) #veify
candidates1 = CH2.retrieve(custlist2)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t4)
# candidates4 = BAv1.retrieve(candidates1[['customer_id','article_id']], t5)
# candidates4 = candidates4.rename(columns={"score_basket_rank0": "score_basket_rank1", "basket_rank0": "basket_rank1" })

ts1 = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=-1)
candidates3 = ts1.retrieve(custlist2)

# candidates = cudf.concat([candidates1, candidates2, candidates3]).fillna(0)
candidates = candidates1.copy()
for can in [candidates2, candidates3]:#, candidates4]:
    candidates = candidates.merge(can, on=['customer_id', 'article_id'], how='outer')
candidates = candidates.fillna(0)

candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)
#----------------------------------------------------------------------


In [ ]:
#creating the 'candidates4' file to save memory - future will be by name and from the class
#till finilize of age strgy with file (also need un-# the customer load and transfer down)
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)
custlist2 = sample_sub.cusid.unique().values_host
tagv1 = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=-1)
candidates4 = tagv1.retrieve(custlist2)
candidates4['score_top_age_group1w_15a'] = candidates4['score_top_age_group1w_15a'].astype('int16')
candidates4.to_csv('candidates4sub.csv', chunksize=1_000_000) 

In [8]:
# gc.collect()
del trainfin
del recalls
del negdf
del negdf1
del negdf2
del negdf3
del negdf4
del negdf5
del negdf7
# gc.collect()

del train, valid, week_recall #can move to before model training
# gc.collect()

del CH, BA1, ts, ts2, tag
gc.collect()

#winner - Final
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
# sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)
sample_sub['cusid'] = sample_sub['cusid'].astype('int32')

#generate candidates
custlist2 = sample_sub.cusid.unique().values_host

CH2 = CustomerHistory2(trans, 4, 15, -1) 
candidates1 = CH2.retrieve(custlist2)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t4)
candidates5 = BAv1.retrieve(candidates1[['customer_id','article_id']], t5) 
candidates2 = cudf.concat([candidates2, candidates5])
candidates2 = candidates2.drop_duplicates(['customer_id', 'article_id'])

ts1 = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=-1)
candidates3 = ts1.retrieve(custlist2)

ts2 = TopSales2(trans, n_weeks=10, n_articles=10, valid_week=0) #valid_week=0 maybe better cause the upper one alreadt taking last week
candidates7 = ts2.retrieve(custlist2)

#loading and adjusting candidtes4 to save memory (file was strgy with same parameters)
candidates4 = cudf.read_csv(path_reduce/'candidates4sub.csv', index_col=0)
candidates4['article_id'] = candidates4['article_id'].astype('int32')
candidates4['customer_id'] = candidates4['customer_id'].astype('int32') #----new to solve for all cands
candidates4['score_top_age_group1w_15a'] = candidates4['score_top_age_group1w_15a'].astype('int16')
candidates4['top_age_group1w_15a'] = candidates4['top_age_group1w_15a'].astype('int8')
# tagv1 = TopAgeGroup(trans, customers, n_weeks=1, n_articles=15, valid_week=-1)
# candidates4 = tagv1.retrieve(custlist2)

candidates = candidates1.copy()
for can in [candidates2, candidates3, candidates4, candidates7]:
    candidates = candidates.merge(can, on=['customer_id', 'article_id'], how='outer')
candidates = candidates.fillna(0)

strgy_names = [x for x in candidates.columns if 'score' not in x][2:] #prbly can cancel
candidates['strgy_count'] = candidates[strgy_names].sum(axis=1).astype('int8')

candidates['valid_week'] = np.int8(-1) 

del trans
del candidates1
del candidates2
del candidates3
del candidates4
del candidates5
del candidates7
del custlist2
del custlist
del t4
del t5
# gc.collect()

del CH2, BAv1, ts1, ts2
gc.collect()

candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
)
del customers
# gc.collect()

candidates =(
    candidates
    .merge(articles2f, on = ('article_id'))
)
del articles2f

#winner 
preds = []
batch_size = 1_000_000
for bucket in tqdm(range(0, len(candidates), batch_size)):
  outputs = ranker.predict(
      candidates.iloc[bucket: bucket+batch_size]
      .drop(columns = ['customer_id', 'article_id'])
      .to_pandas()
      )
  preds.append(outputs)

preds = np.concatenate(preds)
candidates['preds'] = preds
preds = candidates[['customer_id', 'article_id', 'preds']]

del candidates
gc.collect()

# tolist and map(func) not working well with cudf - probably can chnage the solution sort, cut top 12, convert, list to make it much faster and with cudf
preds = preds.to_pandas()
#adjusting format to top 12
preds.sort_values(['customer_id', 'preds'], ascending=False, inplace = True) 
preds = ( 
    preds
    .groupby('customer_id')[['article_id']]
    .aggregate(lambda x: x.tolist())
)

sample_sub = sample_sub.to_pandas()
sample_sub = sample_sub.merge(preds, left_on='cusid', right_index=True, how='left')

def forcon(pr):
    forc = []
    for i in pr:
#         forc.append('0' + str(DHA.mapdict['idx_to_article_id_dict'][i]))
        forc.append('0' + str(idx_to_article_id_dict[i]))
    return ' '.join(forc)

sample_sub['prediction2'] = sample_sub.article_id.map(forcon)

sample_sub = sample_sub.drop(columns=['prediction', 'cusid' ,'article_id'])
sample_sub = sample_sub.rename(columns={'prediction2': 'prediction'})
sample_sub['prediction'] = sample_sub['prediction'].apply(lambda x: x[:131])
sample_sub.to_csv('f5stgyf.csv.gz', index=False)
# !kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f 'data/subs/{sub_name}.csv.gz' -m {sub_name}

100%|██████████| 57/57 [13:20<00:00, 14.04s/it]


In [ ]:
#old basket score dict
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])

#generate candidates
CH2 = CustomerHistory(trans, 4, 15, -1) #veify
custlist2 = sample_sub.cusid.unique().values_host
candidates1 = CH2.retrieve(custlist2)
candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)
candidates2 = candidates1.copy()
candidates2 = candidates2.loc[candidates2.score.values>0]    

candidates2['score'] = candidates2.article_id.map(new_score) #change to real acore check if its really taht low
candidates2['article_id'] = candidates2.article_id.map(new_pairs)


# candidates2['article_id'] = candidates2.article_id.map(new_pairs)
# candidates2['score'] = candidates2.article_id.map(new_score)

candidates2['strtgy2'] = np.int8(1)


candidates = cudf.concat([candidates1, candidates2]).fillna(0)
candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#old manual adjusting names----------------------------------------------------------------------------------
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
# sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)

#generate candidates
custlist2 = sample_sub.cusid.unique().values_host
CH2 = CustomerHistory2(trans, 4, 15, -1) #veify
candidates1 = CH2.retrieve(custlist2)
# candidates1['score'] = scaler.fit_transform(candidates1.score.values_host.reshape(-1, 1)).flatten()
candidates1['strtgy1'] = np.int8(1)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t3)
candidates2['strtgy2'] = np.int8(1)

ts1 = TopSales(trans, n_weeks=1, n_articles=30, valid_week=-1)
candidates3 = ts1.retrieve(custlist2)
candidates3['strtgy3'] = np.int8(1)
candidates3['score'] = np.int8(0)

candidates = cudf.concat([candidates1, candidates2, candidates3]).fillna(0)
candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)

#old before adding candiate4----------------------------------------------------------------------------
#old final with seperated scores
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
# sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])
sample_sub['cusid'] = sample_sub.customer_id.map(customer_id_to_idx_dict)

#generate candidates
custlist2 = sample_sub.cusid.unique().values_host
CH2 = CustomerHistory2(trans, 4, 15, -1) #veify
candidates1 = CH2.retrieve(custlist2)

BAv1 = basket_analysis(trans) #2nd time calculte the pair, need to add loading option
candidates2 = BAv1.retrieve(candidates1[['customer_id','article_id']], t4)
# candidates4 = BAv1.retrieve(candidates1[['customer_id','article_id']], t5)
# candidates4 = candidates4.rename(columns={"score_basket_rank0": "score_basket_rank1", "basket_rank0": "basket_rank1" })

ts1 = TopSales2(trans, n_weeks=1, n_articles=30, valid_week=-1)
candidates3 = ts1.retrieve(custlist2)

# candidates = cudf.concat([candidates1, candidates2, candidates3]).fillna(0)
candidates = candidates1.copy()
for can in [candidates2, candidates3]:#, candidates4]:
    candidates = candidates.merge(can, on=['customer_id', 'article_id'], how='outer')
candidates = candidates.fillna(0)

candidates['valid_week'] = -1 #check , and also for np.int8(-1)

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)
#----------------------------------------------------------------------


In [ ]:
#old 1 strgy
def weeksplit(df, week_number=0):
    valid = df.loc[df.week==week_number]
    train = df.loc[df.week>week_number]
    return train, valid

# test_week = 0
# trans, test = weeksplit(trans, test_week)

recalls = []
for valid_week in [0, 1, 2]:
    train, valid = weeksplit(trans, valid_week)
    custlist = valid.customer_id.unique().values_host #maybe take also for valid?
    CH = CustomerHistory(trans, 4, 15, valid_week)
    negdf = CH.retrieve(custlist)
    negdf = negdf.drop_duplicates()
    buys = valid[['customer_id', 'article_id']].drop_duplicates()
    buys['label'] = 1
    week_recall = negdf.merge(buys, on=['customer_id', 'article_id'], how='outer').fillna({'label':0})
    week_recall['valid_week'] = valid_week
    recalls.append(week_recall)

#added valid score
trainfin = cudf.concat(recalls)
trainfin = trainfin[trainfin['score'].notna()] #only recalls no positive from weekly valid

# trainfin = trainfin.fillna({'score':-1})

trainfin = (trainfin
     .merge(customers, on='customer_id')
     .merge(articles, on='article_id')
)

trainfin =  trainfin[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','label', 'score']]

trainfin = trainfin.to_pandas() #check if cuold avoid
gc.collect()

#training lightgbm ranker ---new
#added valid week sort and gruup
from lightgbm.sklearn import LGBMRanker

ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    max_depth=7,
    n_estimators=300,
    importance_type='gain',
    verbose=10
)

# trainfin.sort_values(['customer_id', 'valid_week'], inplace = True)
# train_baskets = trainfin.groupby(['customer_id', 'valid_week'])['article_id'].count().values

trainfin.sort_values(['valid_week', 'customer_id'], inplace = True)
train_baskets = trainfin.groupby(['valid_week', 'customer_id'])['article_id'].count().values

ranker = ranker.fit(
    trainfin.drop(columns = [ 'customer_id', 'article_id', 'label']),
    trainfin.pop('label'),
    group=train_baskets,
)

In [ ]:
#old 1 stgy part 2
#loading data to predict
sample_sub = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
sample_sub['cusid'] = sample_sub.customer_id.map(DHC.mapdict['customer_id_to_idx_dict'])

#generate candidates
CH2 = CustomerHistory(trans, 4, 12, -1) #veify
custlist2 = sample_sub.cusid.unique().values_host
candidates = CH2.retrieve(custlist2)
candidates['valid_week'] = -1 #check

del trans
gc.collect()


#add features and addjusting to model
candidates = (
    candidates
    .merge(customers, on = ('customer_id'))
    .merge(articles, on = ('article_id'))
)


candidates =  candidates[['valid_week', 'customer_id', 'article_id','FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age','product_type_name', 'product_group_name',
       'graphical_appearance_name', 'colour_group_name',
       'perceived_colour_value_name', 'perceived_colour_master_name',
       'department_name', 'index_name', 'index_group_name', 'section_name',
       'garment_group_name','score']]

#predicting

candidates = candidates.to_pandas()

preds = []
batch_size = 1_000_000
for bucket in tqdm(range(0, len(candidates), batch_size)):
  outputs = ranker.predict(
      candidates.iloc[bucket: bucket+batch_size]
      .drop(columns = ['customer_id', 'article_id'])
      )
  preds.append(outputs)
    
preds = np.concatenate(preds)
candidates['preds'] = preds
preds = candidates[['customer_id', 'article_id', 'preds']]

del candidates
gc.collect()

#adjusting format to top 12
preds.sort_values(['customer_id', 'preds'], ascending=False, inplace = True)
preds = (
    preds
    .groupby('customer_id')[['article_id']]
    .aggregate(lambda x: x.tolist())
)

sample_sub = sample_sub.to_pandas()

sample_sub = sample_sub.merge(preds, left_on='cusid', right_index=True, how='left')

gc.collect()

def forcon(pr):
    forc = []
    for i in pr:
        forc.append('0' + str(DHA.mapdict['idx_to_article_id_dict'][i]))
    return ' '.join(forc)

sample_sub['prediction2'] = sample_sub.article_id.map(forcon)
sample_sub = sample_sub.drop(columns=['prediction', 'cusid' ,'article_id'])
sample_sub = sample_sub.rename(columns={'prediction2': 'prediction'})
sample_sub.to_csv('fmodelnewpro55w.csv.gz', index=False)
# !kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f 'data/subs/{sub_name}.csv.gz' -m {sub_name}


## Prepare validation for submission format

In [ ]:
#copied from up of the final soultion - not in use for final solution
#########currently not in use
# import matplotlib.pyplot as plt
import warnings #to remove warning (long time wanted to see )
# warnings.filterwarnings('ignore') check if this remove the warning
# import seaborn as sns
# from PIL import Image (in chris basket there is some discussio nabout loaidng images better)
tqdm.pandas()
print(f"pandas version: {pd.__version__}")
# pd.options.display.max_colwidth = 200
import pdb
pdb.set_trace()

In [ ]:
#looks like submittion of older solution 
#Series with map customer and what he bought in the valid week
# actual = valid.to_pandas().groupby('customer_id').article_id.apply(list)
#convert to string of articles like the submission format
def to_sub(row):
    s=''
    for l in row:
        s=s+'0'+ str(l) + ' '
    return s.strip()

In [ ]:
#same us the upper one only for first fold (validf1)
actual = validf1.to_pandas().groupby('customer_id').article_id.apply(list)
actualsub = actual.apply(to_sub) 

In [ ]:
#same us the upper one only for 2nd fold (validf2)
actual = validf2.to_pandas().groupby('customer_id').article_id.apply(list)
actualsub = actual.apply(to_sub) 

In [ ]:
#same us the upper one only for 3rd fold (validf3)
actual = validf3.to_pandas().groupby('customer_id').article_id.apply(list)
actualsub = actual.apply(to_sub) 

## Check idea with internal cv

In [ ]:
med_big, mean_big, friq_big  = top_weekly(trainf3)

In [ ]:
def to_cv(submit, big, valid):
    top12 = '0' + big[:12].index.to_pandas().astype('str') 
    top12 = ' '.join(top12.to_list()) 
    submit['prediction'] = top12
    submit['customer_id'] = submit.customer_id.map(id_to_index_dict)
    submitmk = submit.to_pandas().merge(valid, left_on='customer_id', right_index=True)
    return mapk(valid.str.split(' '), submitmk.prediction.str.split(' '))

In [ ]:
submit = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
to_cv(submit, med_big, actualsub)

#### solution2 - run weekly stats on seperate gender

In [ ]:
def by_gen(df, artiscore):
    df = df.merge(artiscore, on='article_id')
    ###can do one time transfer
    custsex = df.groupby('customer_id').index_group_name.sum()
    custsex = custsex>0
    df = df.merge(custsex, left_on='customer_id', right_index=True)
    df.drop(columns=['index_group_name_x'], inplace=True)
    ###can do one time transfer#########
    df.rename({'index_group_name_y': 'is_female'}, axis=1, inplace=True)
    df['week_n'] = df.t_dat.to_pandas().dt.isocalendar().week
    df['year'] = df.t_dat.dt.year
    sold_per_week = df.groupby(['is_female', 'year', 'week_n', 'article_id']).sales_channel_id.count().reset_index() 
    sold_per_week.rename({'sales_channel_id': 'Total_sold'}, axis=1, inplace=True)
    return sold_per_week

In [ ]:
sold_per_week = by_gen(trainf1, articles)

In [ ]:
top12f = sold_per_week.loc[sold_per_week.is_female].groupby('article_id').Total_sold.mean().nlargest(12)
top12f = '0' + top12f[:12].index.to_pandas().astype('str') 
top12f = ' '.join(top12f.to_list())

top12m = sold_per_week.loc[~sold_per_week.is_female].groupby('article_id').Total_sold.mean().nlargest(12)
top12m = '0' + top12m[:12].index.to_pandas().astype('str') 
top12m = ' '.join(top12m.to_list())

In [ ]:
submit = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
submit2 = submit.merge(custsex, how='left', left_index=True, right_index=True)
submit2.index_group_name.fillna(True, inplace=True)
submit2.loc[submit2.index_group_name, 'prediction'] = top12f
submit2.loc[~submit2.index_group_name, 'prediction'] = top12m
submit2.drop(columns=['index_group_name'], inplace=True)
submit2 = submit2.to_pandas()

In [ ]:
#do function so can use the submit2 to LB
submit2['customer_id'] = submit2.customer_id.map(id_to_index_dict)
submitmk = submit2.merge(actualsub, left_on='customer_id', right_index=True)
mapk(actualsub.str.split(' '), submitmk.prediction.str.split(' '))

####  3 - most popular only in the last week *****need validate

In [ ]:
submit2 = cudf.read_csv(path/'sample_submission.csv', low_memory=False)
day = np.timedelta64(1,'D')
w1 = train.loc[train.t_dat>train.t_dat.max() - 7*day]
top12w1 = w1.article_id.value_counts()[:12].index
top12w1 = '0' + top12w1.astype('str') 
top12w1 = ' '.join(top12w1.to_pandas().to_list()) 
submit2['prediction'] = top12w1

## Check result in LB

#### solution1

In [ ]:
#all the train, no splits, same pipeline
med_big, mean_big, friq_big  = top_weekly(train)

In [ ]:
#generating file for submission (return the DF)
def to_submit(submit, big, name='submit'):
    top12 = '0' + big[:12].index.to_pandas().astype('str') 
    top12 = ' '.join(top12.to_list()) 
    submit['prediction'] = top12
    name = name + '.csv'
    submit.to_csv(name, index=False) 
    return submit

#### solution 2

In [ ]:
#offcourse - use the submit2 without running the CV
submit2.to_csv('genderlbl.csv', index=False)

# EDA

In [ ]:
# EDA on the images - cool notebook from other competation
https://www.kaggle.com/code/ishandutta/v7-shopee-indepth-eda-one-stop-for-all-your-needs/notebook
#PCA seasnality anaysis (and plotly.express) but his newbiw not sure how good 
https://www.kaggle.com/code/lichtlab/h-m-data-deep-dive-chap-1-understand-article/notebook

In [ ]:
#reducing dataset size by random sampling (most plots remain the same with 1000 roes samples... stats rules!!!)
toytrain = train.sample(frac=0.001, axis=0 )
display(toytrain)
toytrain.info()

In [ ]:
#fast plot graphs directly from pandas trick
plt.figure(figsize=(16, 9))
toytrain.to_pandas().groupby(["t_dat"])["article_id"].count().plot()

In [ ]:
#nicer formating for pandas tables
toyarticles = toyarticles.to_pandas()
toyarticles.describe().T.sort_values(by='std' , ascending = False)\
.style.background_gradient(cmap='GnBu')\
.bar(subset=["max"], color='#F8766D')\
.bar(subset=["mean",], color='#00BFC4')
                    

In [ ]:
import nltk
from wordcloud import WordCloud #, STOPWORDS internal set used by the finction by default

words_raw = articles['detail_desc'].dropna().to_pandas().apply(nltk.word_tokenize)
bag_of_words = " ".join(words_raw.explode())

fig, ax1 = plt.subplots(figsize=(8,6))
wordcloud = WordCloud( background_color="white", height=300, contour_width=3).generate(bag_of_words) #stopwords=stopwords
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
pd.options.display.max_rows = None 
pd.options.display.max_columns = None
# pd.set_option('display.float_format', '{:.4f}'.format) 
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
#build custom display function for multiple images
top10 = toytrain.groupby('article_id').sales_channel_id.count().nlargest(10).index
f, ax = plt.subplots(2, 5, figsize=(20, 10))
ax = ax.flatten() #to plot on more than one row with no double indx
for aid, a in zip(top10.to_arrow(), ax)
    faid = '0' + str(aid)
    fp = path/'images'/faid[:3]/(faid+'.jpg')
    if fp.exists():
        im = Image.open(fp)
        a.imshow(im)   
        a.set_title(f'ID: {faid}')
        a.set_xticks([], [])
        a.set_yticks([], [])
        a.grid(False)
        a.set_xlabel('Nada', fontsize=10)

In [ ]:
import matplotlib.ticker as ticker

f, ax = plt.subplots(figsize=(10, 5))
ax = sns.histplot(toycust.to_pandas(), x='age', bins=toycust.age.nunique(), color='magenta', stat='percent')
ax.yaxis.set_major_formatter(ticker.PercentFormatter())
median = toycust['age'].median()
ax.axvline(x=median, color="green", ls="--")
ax.text(median, 3.5, 'median: {}'.format(round(median,1)), rotation='vertical', ha='right')
ax.text(40, 8, 'Distribution of customers age', color='black', fontsize=10, ha='left', va='bottom', weight='bold', style='italic')
plt.show()

In [ ]:
toytrain['week_n'] = toytrain.t_dat.to_pandas().dt.isocalendar().week
toytrain['year'] = toytrain.t_dat.dt.year
sold_per_week = toytrain.groupby(['year', 'week_n', 'article_id']).sales_channel_id.count().reset_index()
sold_per_week.rename({'sales_channel_id': 'Total_sold'}, axis=1, inplace=True)

In [ ]:
generalist = ['0706016001', '0706016002', '0372860001', '0610776002', '0759871002', '0464297007', '0372860002', '0610776001', '0399223001', '0706016003', '0720125001', '0156231001'] 
gl = [int(x) for x in generalist[:3]]
mainitems = sold_per_week.loc[sold_per_week.article_id.isin(gl)]
sns.catplot(x="week_n", y="Total_sold", col="year",row = 'article_id',  kind="bar", data=mainitems.to_pandas())

In [ ]:
def mbreak(s):
    return s.to_pandas().sort_values().diff().max().days  
breaks = train.groupby('article_id').t_dat.apply(mbreak)
d=range(30, 720, 30)
for i in tqdm(d):
    print(f'total {(breaks<i).sum()} items {(breaks<i).mean()*100:.2f}% for {i} days')

In [ ]:
f'{breaks.isnull().mean()*100:.2f}%' #4.3% were bought only once

In [ ]:
lastbuy = train.groupby('article_id').t_dat.max()
day = np.timedelta64(1,'D')
maxd = train.t_dat.max()
for i in range(30, 720, 30):
    print(f'{(lastbuy>maxd-i*day).mean()*100:.2f}% bought last time {i*day} ago ')

In [ ]:
#about 80% no longer break than 90 days so we take last 90 days (only 40% items)
#check for same for customer

In [ ]:
cbreak = train.groupby('customer_id').t_dat.apply(mbreak)

In [ ]:
cmacs = toytrain.groupby('customer_id').t_dat.max()

In [ ]:
d = np.dat(1,'D')
dmax = train.t_dat.max()
for i in range(30, 720, 30):
    print(f'{(cmacs>dmax-i*d).mean()}')

In [ ]:
cbreak.sort_values(ascending=False)

In [ ]:
#Checking how much can cut from customer and articles using breaks and last buy data


In [ ]:
#ceck % of customer buying more same item more than one time based on the table above (nbuys >1)
numbuys = train.groupby(['customer_id', 'article_id']).t_dat.nunique().reset_index().rename(columns = {'t_dat':'nbuys'})
cust_max_buys = numbuys.groupby('customer_id').nbuys.max()
cust_max_buys.value_counts(normalize=True)*100 #72% buys dont buy same item in different date

#checking pattern for the multibuyrsn (leave NA for better normalize ) 
multibuys = numbuys.groupby(['customer_id', 'nbuys']).size().to_pandas().unstack()#fill_value=0)
pd.set_option('display.float_format', '{:.4f}'.format)
multibuys[2].value_counts(normalize=True) #46% one case, 20% two case, 10% 3 case

In [ ]:
#ceck % of customer buying more same item more than one time based on the table above (nbuys >1)
numbuys = train.groupby(['customer_id', 'article_id']).t_dat.nunique().reset_index().rename(columns = {'t_dat':'nbuys'})
cust_max_buys = numbuys.groupby('customer_id').nbuys.max()
cust_max_buys.value_counts(normalize=True) #72% buys dont buy same item in different date

In [ ]:
#cancel this two, need functio to diplay images nucely and other to display stats
#show pic and sales data for specific encoded article
def artidata(aid, df, stats=False, path=path):
    faid = '0' + str(aid)
    fp = path/'images'/faid[:3]/(faid+'.jpg')
    if fp.exists():
        im = Image.open(fp)
        im.thumbnail((128,128))
        display(im)
        im.close()
        print(aid)
    if stats:
        tf = df.loc[df.article_id==aid]
        tf = tf.to_pandas()
        sns.countplot(
            x="t_dat", data=tf, 
            order = tf.t_dat.sort_values().values
            )
        _ =plt.xticks(rotation=90)
        plt.show()
        print(aid*2)
        
#show  data for specific encoded customer
def custidata(aid, df, path=path):
    tf = df.loc[df.customer_id==aid] 
    sns.countplot(
        x="t_dat", data=tf.to_pandas(), 
        order = tf.to_pandas().t_dat.sort_values().values)
    _ =plt.xticks(rotation=90)
    plt.show()
    print(aid)     
    
#another reason to cancel this function 
tt = train.groupby('article_id').sales_channel_id.count().sort_values().tail()
ttl = tt.index.values_host
for i in ttl:
    artidata(i, train, stats=True)

In [ ]:
tt = toytrain.groupby('customer_id').sales_channel_id.count().sort_values().tail()
ttl = tt.index.values_host
for i in ttl:
    custidata(i, toytrain)

In [ ]:
#pure cold start - customer with no buy history
#index is the maping so just need index numbers no no in train
cold_start_idx = submit.loc[~submit.index.isin(train.customer_id.values)].index
#9699 cusomers

# Ensembling

Taken from this notebook: https://www.kaggle.com/code/titericz/h-m-ensembling-how-to

In [ ]:
#need to prepare first a DF with all the predictions we want to ensemble and after that adjust the:
#W variable based on the number of models to enemble
#REC list appending all modles 
def cust_blend(dt, W = [1,1,1]):
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12]) #RK = return in for each row, all the avove is done per Series

sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95], axis=1)

In [ ]:
#Write 5/7 tricks learned on DEA from GM notebook
#Summarize all soultion learned by now 
#Explore ML/DL solution (the one LGBM that sort candidtes???)

In [ ]:
cdf = cudf.DataFrame({'c1':['a', 'a', 'b', 'c', 'e', 'f', 'g'], 'c2':[10,  10, 20, 30, 40, 50, 60], 's1':[2, 1,1,1,1,1,1]})
cdf2 = cudf.DataFrame({'c1':['a', 'b', 'c', 'e', 'f', 'g'], 'c2':[10, 2, 30, 4, 50, 6], 's2':[1, 1, 1, 1, 1, 1]})
cdf3 = cudf.DataFrame({'c1':['a', 'b', 'c', 'e', 'f', 'g'], 'c2':[1, 20, 30, 4, 50, 60], 's3':[1,1,1,1,1,1]})
cdlist = [cdf, cdf2, cdf3]